In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, ZeroPadding2D, add
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras import backend as k
import cv2

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/kaggle/Digit Recognizer/data/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/kaggle/Digit Recognizer/data/test.csv")
X_train = (train.iloc[:,1:].values).astype('float32') # all pixel values
y_train = train.iloc[:,0].values.astype(int) # only labels i.e targets digits
X_test = test.values.astype('float32')

In [ ]:
k.clear_session()
X_train = X_train.reshape(-1, 28, 28,1)
X_test = X_test.reshape(-1, 28, 28,1)
X_train=X_train[:12000]
X_test=X_test[:12000]
y_train=y_train[:12000]

In [ ]:
#轉成VGG16需要的格式，要RGB通道(資料太多會爆RAM)
ishape = 224
X_train = [cv2.cvtColor(cv2.resize(i,(ishape,ishape)), cv2.COLOR_GRAY2BGR) for i in X_train]
X_train = np.array(X_train)
# 增加維度(沒用到)
# X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype('float32')
 
X_test  = [cv2.cvtColor(cv2.resize(i,(ishape,ishape)), cv2.COLOR_GRAY2BGR) for i in X_test]
X_test = np.array(X_test)
# X_test  = np.concatenate([arr[np.newaxis] for arr in X_test] ).astype('float32')

In [ ]:
X_train = X_train/255
X_test = X_test/255

y_train = to_categorical(y_train, num_classes = 10)

In [ ]:
random_seed = 42
# Split the train and the validation set for the fitting
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state=random_seed)

In [ ]:
k.clear_session()
model_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(ishape, ishape, 3))
for layers in model_vgg.layers:
    layers.trainable = False
 
model = Flatten()(model_vgg.output)
output = Dense(10, activation='softmax')(model)
model_vgg_pretrain = Model(inputs=model_vgg.input, outputs=output, name='vgg16_pretrain')
# model_vgg_mnist_pretrain.summary()
model_vgg_pretrain.compile(optimizer='adam', loss='categorical_crossentropy',
                                 metrics=['accuracy'])

In [ ]:
# With data augmentation to prevent overfitting
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        data_format='channels_last')  


datagen.fit(X_train)

In [ ]:
#學習率下降
reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_accuracy', 
                              patience=2, 
                              verbose=1)
#儲存最佳模型
model_ckpt = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/kaggle/Digit Recognizer/model/best_trans_model.h5", 
                             monitor="val_accuracy", 
                             save_best_only=True)
#提早終止
earlystop = EarlyStopping(monitor="val_accuracy", 
                          patience=10, 
                          verbose=1
                          )

In [ ]:
# Fit the model
history = model_vgg_pretrain.fit(datagen.flow(X_train,y_train, batch_size=128),
                    # steps_per_epoch=完成一次epoch需要跑多少個batch
                    steps_per_epoch=X_train.shape[0]//128,
                    validation_data=(X_val,y_val),
                    epochs = 200,
                    callbacks=[earlystop, model_ckpt,reduce_lr],
                    verbose = 1)

Epoch 1/200
79/79 [==============================] - 95s 1s/step - loss: 0.5099 - accuracy: 0.8576 - val_loss: 0.1055 - val_accuracy: 0.9756
Epoch 2/200
79/79 [==============================] - 93s 1s/step - loss: 0.1204 - accuracy: 0.9701 - val_loss: 0.0747 - val_accuracy: 0.9783
Epoch 3/200
79/79 [==============================] - 92s 1s/step - loss: 0.0927 - accuracy: 0.9759 - val_loss: 0.0585 - val_accuracy: 0.9828
Epoch 4/200
79/79 [==============================] - 91s 1s/step - loss: 0.0703 - accuracy: 0.9828 - val_loss: 0.0538 - val_accuracy: 0.9817
Epoch 5/200
79/79 [==============================] - ETA: 0s - loss: 0.0602 - accuracy: 0.9847
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
79/79 [==============================] - 91s 1s/step - loss: 0.0602 - accuracy: 0.9847 - val_loss: 0.0536 - val_accuracy: 0.9811
Epoch 6/200
79/79 [==============================] - 92s 1s/step - loss: 0.0483 - accuracy: 0.9887 - val_loss: 0.0489 - val_accuracy